# Part 6.1 - Prepare Dataframe
---

### Papers Past Topic Modeling
<br/>


Ben Faulks - bmf43@uclive.ac.nz

Xiandong Cai - xca24@uclive.ac.nz

Yujie Cui - ycu23@uclive.ac.nz

In [1]:
import os, sys, subprocess
sys.path.insert(0, '../utils') # for import customed modules
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import *
from utils_load import conf_pyspark, load_dataset

# intiate PySpark
sc, spark = conf_pyspark()

sc

[('spark.driver.port', '38085'),
 ('spark.app.name', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1547930702242'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', '192.168.1.207'),
 ('spark.driver.memory', '62g'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.cores', '6'),
 ('spark.driver.maxResultSize', '4g')]


<SparkContext master=local[*] appName=local>

## 1 Prepare Dataframe for Full Dataset

**After training the full dataset, we got the topic words list and the doc-topic matrix, now we load those data to generate the dataframe for analysys and visualization.**

### 1.1 Load data

**Load full dataset and extract metadata ("id", "region" and "date"):**

In [2]:
df_dataset = load_dataset('dataset', spark).select(F.col('id').alias('id_'), F.col('region'), F.col('date'))

**Load topic words list:**

In [3]:
path = r'../5-model/model_train/topicKeys.txt'

data_schema = StructType([
    StructField('topic', IntegerType()),
    StructField('weight', FloatType()),
    StructField('words', StringType())
])

df_topics = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "false")
    .option("delimiter", "\t")
    .schema(data_schema)
    .load(path)
)

In [4]:
print('Shape of dataframe: ({}, {})'.format(df_topics.count(), len(df_topics.columns)))
df_topics.show(20, False)

Shape of dataframe: (250, 3)
+-----+-------+---------------------------------------------------------------------------------+
|topic|weight |words                                                                            |
+-----+-------+---------------------------------------------------------------------------------+
|0    |0.01008|mr party government minister labour sir ward joseph country massey               |
|1    |0.01015|bank cent loan zealand london banks money exchange rate deposits                 |
|2    |0.00692|company apply freight steam passage zealand passengers shipping steamers agents  |
|3    |0.03237|man people men public law good fact make things state                            |
|4    |0.00528|fruit apples pears case potatoes plums peaches oranges poultry lb                |
|5    |0.04785|good time year made week great work present day weather                          |
|6    |0.00788|napier hastings bay hawke waipawa dannevirke woodville danevirke mr tarada

**Load doc-topic matrix:**

In [5]:
path = r'../5-model/model_train/docTopics.txt'

#topic_number = 500
topic_number = 250

# generate new column names
columns = [str(x) for x in list(range(topic_number))]
columns.insert(0, 'id')
columns.insert(0, 'index')

# load data
df_doctopic = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "true")
    .option("delimiter", "\t")
    #.schema(data_schema)
    .load(path)
)

# change columns name and drop # column which is table index and useless
df_doctopic = df_doctopic.toDF(*columns)

In [6]:
print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

# show top 5 row
df_doctopic.limit(5).toPandas().head()

Shape of dataframe: (160140, 252)


,index,id,0,1,2,3,4,5,6,7,...,240,241,242,243,244,245,246,247,248,249
0,0,1854232,0.000063,0.000063,0.187764,0.000203,0.000033,0.000299,0.000049,0.000066,...,0.000112,0.000055,0.000071,0.000042,0.000048,0.000085,0.000069,0.000047,0.000065,0.000028
1,1,1854244,0.000146,0.000147,0.000101,0.000470,0.000077,0.000695,0.000115,0.000153,...,0.000261,0.000127,0.000165,0.000097,0.000111,0.000198,0.000161,0.000108,0.000151,0.000065
2,2,1854262,0.000063,0.000064,0.000044,0.000204,0.000033,0.000301,0.000050,0.000066,...,0.000113,0.000055,0.000071,0.000042,0.000048,0.000086,0.000070,0.000047,0.000066,0.000028
3,3,1854275,0.000063,0.000063,0.068874,0.000203,0.000033,0.000299,0.000049,0.000066,...,0.000112,0.000055,0.000071,0.000042,0.000048,0.000085,0.000069,0.000047,0.000065,0.000028
4,4,1854588,0.000109,0.000109,0.000075,0.000349,0.000057,0.000516,0.000085,0.000113,...,0.689759,0.000094,0.000122,0.000072,0.000082,0.000147,0.000119,0.000080,0.000112,0.000049


**In above dataframe, "index" column is the row number, "id" column is the sample/document/text id, the same with "id" in dataset, "0" to "249" columns are the weight of each topic.**

### 1.2 Add Dominant Topics Column

**Find dominant topic of each document:**

In [7]:
# https://stackoverflow.com/questions/46819405/how-to-get-the-name-of-column-with-maximum-value-in-pyspark-dataframe

def argmax(cols, *args):
    return [c for c, v in zip(cols, args) if v == max(args)][0]

def find_dominant_topic(df):
    """
    find the dominant topic of each sample/row/document
    input: dataframe of weight of each topic
    output: the dominant topic number dataframe
    """
    argmax_udf = lambda cols: F.udf(lambda *args: argmax(cols, *args), StringType())
    return (df
            .withColumn('dominant',argmax_udf(df.columns[2:])(*df.columns[2:]))
            .select(F.col('index').alias('index_'), F.col('dominant')))

df_dominant = find_dominant_topic(df_doctopic)

# add the df_dominant to doc-topic matrix
df_doctopic = (df_doctopic
               .join(df_dominant, df_doctopic.index == df_dominant.index_)
               .drop('index_')
               .orderBy('index'))

In [8]:
print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

df_doctopic.limit(5).toPandas().head()

Shape of dataframe: (160140, 253)


,index,id,0,1,2,3,4,5,6,7,...,241,242,243,244,245,246,247,248,249,dominant
0,0,1854232,0.000063,0.000063,0.187764,0.000203,0.000033,0.000299,0.000049,0.000066,...,0.000055,0.000071,0.000042,0.000048,0.000085,0.000069,0.000047,0.000065,0.000028,2
1,1,1854244,0.000146,0.000147,0.000101,0.000470,0.000077,0.000695,0.000115,0.000153,...,0.000127,0.000165,0.000097,0.000111,0.000198,0.000161,0.000108,0.000151,0.000065,196
2,2,1854262,0.000063,0.000064,0.000044,0.000204,0.000033,0.000301,0.000050,0.000066,...,0.000055,0.000071,0.000042,0.000048,0.000086,0.000070,0.000047,0.000066,0.000028,200
3,3,1854275,0.000063,0.000063,0.068874,0.000203,0.000033,0.000299,0.000049,0.000066,...,0.000055,0.000071,0.000042,0.000048,0.000085,0.000069,0.000047,0.000065,0.000028,211
4,4,1854588,0.000109,0.000109,0.000075,0.000349,0.000057,0.000516,0.000085,0.000113,...,0.000094,0.000122,0.000072,0.000082,0.000147,0.000119,0.000080,0.000112,0.000049,240


### 1.3 Add Metadata Column

**Here we only add "region" and "year" column as metadata. Since the full dataset has the "date" column, the accuracy of time could achieve "day" level, but currently we analyze data by "year" not by "day" or other unit, so for convenience we directly convert date to year.**

In [9]:
df_doctopic = (df_doctopic
               .join(df_dataset, df_doctopic.id == df_dataset.id_)
               .withColumn('year', F.date_format('date', 'yyyy'))
               .drop('id_')
               .drop('date')
               .orderBy('index'))

In [10]:
print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

df_doctopic.limit(5).toPandas().head()

Shape of dataframe: (160140, 255)


,index,id,0,1,2,3,4,5,6,7,...,243,244,245,246,247,248,249,dominant,region,year
0,0,1854232,0.000063,0.000063,0.187764,0.000203,0.000033,0.000299,0.000049,0.000066,...,0.000042,0.000048,0.000085,0.000069,0.000047,0.000065,0.000028,2,Auckland,1863
1,1,1854244,0.000146,0.000147,0.000101,0.000470,0.000077,0.000695,0.000115,0.000153,...,0.000097,0.000111,0.000198,0.000161,0.000108,0.000151,0.000065,196,Auckland,1863
2,2,1854262,0.000063,0.000064,0.000044,0.000204,0.000033,0.000301,0.000050,0.000066,...,0.000042,0.000048,0.000086,0.000070,0.000047,0.000066,0.000028,200,Auckland,1863
3,3,1854275,0.000063,0.000063,0.068874,0.000203,0.000033,0.000299,0.000049,0.000066,...,0.000042,0.000048,0.000085,0.000069,0.000047,0.000065,0.000028,211,Auckland,1863
4,4,1854588,0.000109,0.000109,0.000075,0.000349,0.000057,0.000516,0.000085,0.000113,...,0.000072,0.000082,0.000147,0.000119,0.000080,0.000112,0.000049,240,Canterbury,1887


### 1.4 Dominant Topics Dataframe

**It is hard to plot the doc-topic matrix (high dimension), we need transform it to extract or reduce features. First we generate dominant topics dataframe which could be used to reveal the relationship between dominant topics and region/year.**

In [11]:
df_dominant = (df_doctopic
               .join(df_topics, df_dominant.dominant == df_topics.topic)
               .select(F.col('id'), 
                       F.col('region'), 
                       F.col('year'), 
                       F.col('dominant'), 
                       F.col('words')))

In [12]:
print('Shape of dataframe: ({}, {})'.format(df_dominant.count(), len(df_dominant.columns)))

df_dominant.limit(5).show(5, False)

Shape of dataframe: (160140, 5)
+-------+----------+----+--------+--------------------------------------------------------------------------------+
|id     |region    |year|dominant|words                                                                           |
+-------+----------+----+--------+--------------------------------------------------------------------------------+
|1854232|Auckland  |1863|2       |company apply freight steam passage zealand passengers shipping steamers agents |
|1854244|Auckland  |1863|196     |business public street begs attention notice mr inform support patronage        |
|1854262|Auckland  |1863|200     |life man world great years english work men people time                         |
|1854275|Auckland  |1863|211     |goods street stock prices large wholesale cash orders general advertisements    |
|1854588|Canterbury|1887|240     |french government russia paris troops france british turkish russian turkey     |
+-------+----------+----+--------+------

**Save dataset for later use:**

In [14]:
path = r'./train/domTopics'

df_dominant.write.csv(path, mode='overwrite', compression='gzip')

In [18]:
%%bash -s "$path"

cat $1/*.csv.gz > $1/domTopics.csv.gz

gunzip $1/domTopics.csv.gz

### 1.5 Average Weight Topics Dataframe

**Beside dominant topics dataframe, we could calculate average weight of each topic in a year and create dataframe for it, which could be used to reveal the weight variety of each topic as time goes on.**